In [82]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import matplotlib.image as mpimg
import matplotlib as mpl

# Import API key
from api_keys import g_key

In [83]:
# open games.csv from raw_data folder
games = '../raw_data/games.csv'
games_df = pd.read_csv(games)
# make a list of current season games ID's
games_df.head()
games_id = list(games_df['GAME_ID'])
print(f"In season 2019-2020 prior to COVID-19 were played: {len(games_id)} games")

In season 2019-2020 prior to COVID-19 were played: 602 games


In [84]:
# open games_details.csv from raw_data folder
games_det = '../raw_data/games_details.csv'
games_det = pd.read_csv(games_det)
games_det.head()


,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,21900895,1610612749,MIL,Milwaukee,202083,Wesley Matthews,F,NaN,27:08,3.0,...,4.0,4.0,8.0,2.0,2.0,0.0,0.0,0.0,8.0,11.0
1,21900895,1610612749,MIL,Milwaukee,203507,Giannis Antetokounmpo,F,NaN,34:55,17.0,...,2.0,18.0,20.0,6.0,1.0,0.0,3.0,2.0,41.0,22.0
2,21900895,1610612749,MIL,Milwaukee,201572,Brook Lopez,C,NaN,26:25,4.0,...,2.0,5.0,7.0,0.0,0.0,3.0,0.0,2.0,16.0,16.0
3,21900895,1610612749,MIL,Milwaukee,1628978,Donte DiVincenzo,G,NaN,27:35,1.0,...,1.0,6.0,7.0,5.0,0.0,1.0,2.0,0.0,2.0,14.0
4,21900895,1610612749,MIL,Milwaukee,202339,Eric Bledsoe,G,NaN,22:17,2.0,...,1.0,0.0,1.0,2.0,1.0,0.0,3.0,2.0,4.0,6.0


In [86]:
# compare games id values in games_det data frame with values of games_id list
games_det.isin({'GAME_ID': games_id})
# keep only current season games id's in games_det dataframe 
games_det_new = games_det[games_det['GAME_ID'].isin(games_id)]
# export to csv om py_data folder for further use
games_det_new.to_csv('../py_data/games_details_2020season.csv',index=False, header=True)
games_det_new.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,21900895,1610612749,MIL,Milwaukee,202083,Wesley Matthews,F,NaN,27:08,3.0,...,4.0,4.0,8.0,2.0,2.0,0.0,0.0,0.0,8.0,11.0
1,21900895,1610612749,MIL,Milwaukee,203507,Giannis Antetokounmpo,F,NaN,34:55,17.0,...,2.0,18.0,20.0,6.0,1.0,0.0,3.0,2.0,41.0,22.0
2,21900895,1610612749,MIL,Milwaukee,201572,Brook Lopez,C,NaN,26:25,4.0,...,2.0,5.0,7.0,0.0,0.0,3.0,0.0,2.0,16.0,16.0
3,21900895,1610612749,MIL,Milwaukee,1628978,Donte DiVincenzo,G,NaN,27:35,1.0,...,1.0,6.0,7.0,5.0,0.0,1.0,2.0,0.0,2.0,14.0
4,21900895,1610612749,MIL,Milwaukee,202339,Eric Bledsoe,G,NaN,22:17,2.0,...,1.0,0.0,1.0,2.0,1.0,0.0,3.0,2.0,4.0,6.0


In [92]:
# open teams.csv from raw_data folder
teams = '../raw_data/teams.csv'
teams_df = pd.read_csv(teams)
# df with selectd columns
teams_conf = teams_df[["TEAM_ID", "Conf"]]
teams_conf.head()

,TEAM_ID,Conf
0,1610612737,East
1,1610612738,East
2,1610612740,West
3,1610612741,East
4,1610612742,West


In [94]:
# Merge teams and games details in order to split conference games
games_det_df = pd.merge(games_det_new, teams_conf, on="TEAM_ID", how='outer')
games_det_df.to_csv('../py_data/games_details_by_conference.csv',index=True, header=True)
games_det_df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,Conf
0,21900895,1610612749,MIL,Milwaukee,202083,Wesley Matthews,F,NaN,27:08,3.0,...,4.0,8.0,2.0,2.0,0.0,0.0,0.0,8.0,11.0,East
1,21900895,1610612749,MIL,Milwaukee,203507,Giannis Antetokounmpo,F,NaN,34:55,17.0,...,18.0,20.0,6.0,1.0,0.0,3.0,2.0,41.0,22.0,East
2,21900895,1610612749,MIL,Milwaukee,201572,Brook Lopez,C,NaN,26:25,4.0,...,5.0,7.0,0.0,0.0,3.0,0.0,2.0,16.0,16.0,East
3,21900895,1610612749,MIL,Milwaukee,1628978,Donte DiVincenzo,G,NaN,27:35,1.0,...,6.0,7.0,5.0,0.0,1.0,2.0,0.0,2.0,14.0,East
4,21900895,1610612749,MIL,Milwaukee,202339,Eric Bledsoe,G,NaN,22:17,2.0,...,0.0,1.0,2.0,1.0,0.0,3.0,2.0,4.0,6.0,East


In [97]:
games_df.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,3/1/2020,21900895,Final,1610612766,1610612749,2019,1610612766,85,0.354,0.900,...,22,47,1610612749,93,0.402,0.762,0.226,20,61,0
1,3/1/2020,21900896,Final,1610612750,1610612742,2019,1610612750,91,0.364,0.400,...,19,57,1610612742,111,0.468,0.632,0.275,28,56,0
2,3/1/2020,21900897,Final,1610612746,1610612755,2019,1610612746,136,0.592,0.805,...,25,37,1610612755,130,0.505,0.650,0.488,27,37,1
3,3/1/2020,21900898,Final,1610612743,1610612761,2019,1610612743,133,0.566,0.700,...,38,41,1610612761,118,0.461,0.897,0.263,24,36,1
4,3/1/2020,21900899,Final,1610612758,1610612765,2019,1610612758,106,0.407,0.885,...,18,51,1610612765,100,0.413,0.667,0.429,23,42,1


In [98]:
teams_df.head()

,LEAGUE_ID,TEAM_ID,Conf,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,East,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,East,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,West,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,East,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,West,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends
